In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "_paper_1"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using MorphoMolNotebooks
using CairoMakie
using JLD2
using Random

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`
Precompiling MorphoMol
  ✓ MorphoMol
  1 dependency successfully precompiled in 3 seconds. 258 already precompiled.
Precompiling MorphoMolNotebooks
  ✓ MorphoMolNotebooks
  1 dependency successfully precompiled in 6 seconds. 283 already precompiled.


generate_transitions (generic function with 1 method)

In [30]:
n = 6
rs = 1.0
eta = 0.475
pf = MorphoMol.Energies.get_prefactors(rs, eta)
σ_t = rs
T_search = MorphoMolNotebooks.get_dispersed_energy(n, rs, pf) / 15.0
iterations = 500000
target_acceptance_rate = 0.5
T_sim = temp_finder(
    n,
    rs,
    pf,
    σ_t,
    T_search, 
    iterations,
    target_acceptance_rate
)

1.2573454048805766

In [31]:
template_centers = [0.0, 0.0, 0.0]
bounds = n / 2.0 + 3.0
x_init = get_random_configuration_without_overlap_in_bounds(n, bounds)
radii = fill(1.0, n)

overlap_slope = 100000.0
overlap_jump = 100000.0

energy(x) = solvation_free_energy_in_bounds(x, radii, rs, pf, overlap_jump, overlap_slope, bounds, 100.0)
perturbation(x) = MorphoMolNotebooks.perturb_single_randomly_chosen(x, σ_t)
temp_r(i) = T_sim

output = Dict{String,Vector}(
    "states" => [],
    "Es" => []
)

sa = MorphoMol.Algorithms.SimulatedAnnealing(energy, perturbation, temp_r)
x, E, alpha = MorphoMol.Algorithms.simulate!(sa, x_init, iterations, output);
alpha

0.275438